In [1]:
import os
import pandas as pd
import numpy as np
from tqdm import tqdm
import seaborn as sns

# Merging cc and loy 

In [2]:
df_cc=pd.read_csv('cc_data.csv',encoding='latin-1')
df_loy=pd.read_csv('loyalty_data.csv',encoding='latin-1')

In [7]:
col=[x for x in df_cc.last4ccnum]
col.extend([y for y in df_loy.loyaltynum])
cc_loy_concat=pd.concat([df_cc,df_loy], axis=0)
cc_loy_concat['num']=[str(x) for x in col]
cc_loy_concat.drop(['last4ccnum','loyaltynum'],axis=1, inplace=True)
cc_loy_concat['Date']=[x.split(' ')[0] for x in cc_loy_concat.timestamp]
cc_loy_concat['Hour']=['' for x in cc_loy_concat.timestamp]
cc_loy_concat['Minute']=['' for x in cc_loy_concat.timestamp]

for i,each in enumerate(cc_loy_concat.timestamp):
    if len(each.split(' '))==1:
        cc_loy_concat.iloc[i,-2]='Not specified'
        cc_loy_concat.iloc[i,-1]='Not specified'
    else:
        cc_loy_concat.iloc[i,-2]=each.split(' ')[-1].split(':')[0]
        cc_loy_concat.iloc[i,-1]=each.split(' ')[-1].split(':')[1]
cc_loy_concat.drop('timestamp',axis=1)
cc_loy_concat.to_csv('cc_loy.csv',header=True,index=False)
#__________________
df_ccloy_merged=pd.read_csv('cc_loy.csv')
df_ccloy_merged.timestamp=[x.split(' ')[0] for x in df_ccloy_merged.timestamp]
df_ccloy_merged['price_count_ratio']=[0 for x in df_ccloy_merged.timestamp]
d_ratio={(k,l):[0,0] for k,l in zip(df_ccloy_merged.timestamp, df_ccloy_merged.location)}
d_ratio

for i,each in enumerate(df_ccloy_merged.timestamp):
    d_ratio[(each,df_ccloy_merged.location[i])][0]+=1
    d_ratio[(each,df_ccloy_merged.location[i])][1]+=df_ccloy_merged.price[i]     #np.sqrt(df_ccloy_merged.price[i])


for item in d_ratio.items():
    df_temp=df_ccloy_merged[(df_ccloy_merged['timestamp']==item[0][0]) & (df_ccloy_merged['location']==item[0][1])]
    for i in df_temp.index:
        #print(item[1][1],item[1][0], item[1][1]/item[1][0])
        df_ccloy_merged.iloc[i,-1]= (item[1][1]/item[1][0])
df_ccloy_merged.to_csv('cc_loy.csv', header=True, index=False)
df_ccloy_merged

,timestamp,location,price,num,Date,Hour,Minute,price_count_ratio
0,01/06/2014,Brew've Been Served,11.34,4795,01/06/2014,07,28,10.946061
1,01/06/2014,Hallowed Grounds,52.22,7108,01/06/2014,07,34,16.308125
2,01/06/2014,Brew've Been Served,8.33,6816,01/06/2014,07,35,10.946061
3,01/06/2014,Hallowed Grounds,16.72,9617,01/06/2014,07,36,16.308125
4,01/06/2014,Brew've Been Served,4.24,7384,01/06/2014,07,37,10.946061
...,...,...,...,...,...,...,...,...
2877,01/19/2014,Katerinas Café,70.22,L8566,01/19/2014,Not specified,Not specified,39.306250
2878,01/19/2014,Guy's Gyros,18.08,L9254,01/19/2014,Not specified,Not specified,38.893636
2879,01/19/2014,Katerinas Café,35.01,L9254,01/19/2014,Not specified,Not specified,39.306250
2880,01/19/2014,Desafio Golf Course,110.02,L6886,01/19/2014,Not specified,Not specified,148.988000


# Merging assign_car and gps_data

In [4]:
df_assign=pd.read_csv('car-assignments.csv')
df_gps=pd.read_csv('gps.csv')

d_assign_id={}
for i,each in enumerate(df_assign.CarID):
    d_assign_id[each]= [df_assign.LastName[i]+' '+df_assign.FirstName[i],
                        df_assign.CurrentEmploymentType[i],
                        df_assign.CurrentEmploymentTitle[i]]
    
df_gps['fullname']=['' for x in range(len(df_gps))] #CurrentEmploymentType	CurrentEmploymentTitle
df_gps['CurrentEmploymentType']=['' for x in range(len(df_gps))]
df_gps['CurrentEmploymentTitle']=['' for x in range(len(df_gps))]
for i,each in tqdm(enumerate(df_gps.id)): #2 minute running
    try:
        df_gps.iloc[i,-3]=d_assign_id[each][0]
        df_gps.iloc[i,-2]=d_assign_id[each][1]
        df_gps.iloc[i,-1]=d_assign_id[each][2]
    except: #possible errors due to missing index 
        pass
    
df_gps['Date']=[x.split(' ')[0] for x in df_gps['Timestamp']] #gps_assign
df_gps['Hour']=[x.split(' ')[1].split(':')[0] for x in df_gps['Timestamp']]
df_gps['Minute']=[x.split(' ')[1].split(':')[1] for x in df_gps['Timestamp']]
df_gps['Second']=[x.split(' ')[1].split(':')[2] for x in df_gps['Timestamp']]   

d_timestamps={k:i+1 for i,k in enumerate(sorted(list(set(df_gps.Timestamp))))}
df_gps['n_timestamp']=[d_timestamps[x] for x in df_gps['Timestamp']]

df_gps.to_csv('gps_assign_merged.csv', index=False, header=True)


685169it [02:22, 4805.18it/s] 


# Bonus: Sunburst formatting

In [19]:
#CORRECTED! (version: date, hour, location, cc_num)
#CC_DATA SUNBURST FORMAT  #the same for loy_data but with hier path==3 (no Hour) and different num name (see loy_data)
df_cc['Date']=[x.split(' ')[0] for x in df_cc.timestamp]
df_cc['Hour']=[x.split(' ')[1] for x in df_cc.timestamp]

df_filt_all=df_cc.groupby(by=['Date','Hour','location','last4ccnum']).sum()

temp_ids=[]
temp_parents=[]
temp_labels=[]
temp_values= []

id_checker=''
for i,index in tqdm(enumerate(df_filt_all.index)):
    if str(index[0])+'_' not in temp_ids:
        temp_ids.append(str(index[0])+'_')
        temp_parents.append('')
        temp_labels.append(str(index[0]))
        #filter value
        temp_df= df_cc[df_cc['Date']==index[0]].groupby(by=['Date']).sum()
        temp_values.append(temp_df['price'].values[0] )
        #print(temp_df['price'].values)

    if str(index[0])+'_'+str(index[1]) not in temp_ids:
        temp_ids.append(str(index[0])+'_'+str(index[1]))
        temp_parents.append(str(index[0])+'_')
        temp_labels.append(str(index[1]))
        #filter value
        temp_df= df_cc[(df_cc['Date']==index[0]) &
                         (df_cc['Hour']==index[1]) ].groupby(by=['Date','Hour']).sum()
        temp_values.append(temp_df['price'].values[0] )
        
    if str(index[0])+'_'+str(index[1])+'_'+str(index[2]) not in temp_ids:
        temp_ids.append(str(index[0])+'_'+str(index[1])+'_'+str(index[2]))
        temp_parents.append(str(index[0])+'_'+str(index[1]))
        temp_labels.append(str(index[2]))
        #filter value
        temp_df= df_cc[(df_cc['Date']==index[0]) &
                         (df_cc['Hour']==index[1]) &
                         (df_cc['location']==index[2]) ].groupby(by=['Date','Hour','location']).sum()
        temp_values.append(temp_df['price'].values[0] )
        
    if str(index[0])+'_'+str(index[1])+'_'+str(index[2])+'_'+str(index[3]) not in temp_ids:
        temp_ids.append(str(index[0])+'_'+str(index[1])+'_'+str(index[2])+'_'+str(index[3]))
        temp_parents.append(str(index[0])+'_'+str(index[1])+'_'+str(index[2]))
        temp_labels.append(str(index[3]))
        
        #filter value
        temp_df= df_cc[(df_cc['Date']==index[0]) &
                         (df_cc['Hour']==index[1]) &
                         (df_cc['location']==index[2]) &
                         (df_cc['last4ccnum']==index[3]) ].groupby(by=['Date','Hour','location']).sum()
        temp_values.append(temp_df['price'].values[0] )
        
 #print(temp_ids, temp_parents, temp_labels)

df_new=pd.DataFrame(columns=['id','label','parent','value'])
df_new['id']=temp_ids
df_new['label']=temp_labels
df_new['parent']=temp_parents
df_new['value']=temp_values
df_new
#df_new.to_csv('cc_hier.csv',index=False,header=True)  

1490it [00:15, 93.37it/s] 


,id,label,parent,value
0,01/06/2014_,01/06/2014,,22539.25
1,01/06/2014_07:28,07:28,01/06/2014_,11.34
2,01/06/2014_07:28_Brew've Been Served,Brew've Been Served,01/06/2014_07:28,11.34
3,01/06/2014_07:28_Brew've Been Served_4795,4795,01/06/2014_07:28_Brew've Been Served,11.34
4,01/06/2014_07:34,07:34,01/06/2014_,52.22
...,...,...,...,...
3959,01/19/2014_20:30_Guy's Gyros,Guy's Gyros,01/19/2014_20:30,31.94
3960,01/19/2014_20:30_Guy's Gyros_4434,4434,01/19/2014_20:30_Guy's Gyros,31.94
3961,01/19/2014_20:51,20:51,01/19/2014_,39.60
3962,01/19/2014_20:51_Guy's Gyros,Guy's Gyros,01/19/2014_20:51,39.60
